Assignment 1: Web scrapping 

imports

In [108]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

make http request to get html document

In [109]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)

construct html parser

In [110]:
soup = BeautifulSoup(response.text, "html.parser")

parse/scrape html content into dataframe

In [111]:
import pandas as pd
df = pd.DataFrame(columns=["PostalCode","Borough","Neighborhood"])
df.set_index("PostalCode", inplace=True)
             
table1 = soup.find('table', class_="wikitable sortable")
trs = table1.findAll('tr')
for i in range(0,len(trs)-1):
    tr1 = trs[i]
    tds = tr1.findAll('td')
    if (len(tds)==3 and tds[1].text!="Not assigned"):
        postalCode=tds[0].text
        borough=tds[1].text
        neighborhood=tds[2].text
        if neighborhood=="Not assigned":
            neighborhood = borough
        else:
            neighborhood = neighborhood.replace("\n","")
        
        try:
            df.loc[postalCode]["Neighborhood"] = df.loc[postalCode]["Neighborhood"] + ", " + neighborhood 
        except: 
            df.loc[postalCode] = [borough, neighborhood] 

print(df.shape)        
df.head()

(103, 2)


,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


Assignment 2: geocoder

add columns to dataframe for lat,lng location

In [112]:
df["Latitude"]=float('nan')
df["Longitude"]=float('nan')
df.dtypes

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

geocoder does not work, so just load the lookup dataset

In [113]:
#Error using geocoder 
#!pip install geocoder
#import geocoder
#geocoder.google('M3A, Toronto, Ontario')
#error <[REQUEST_DENIED] Google - Geocode [empty]>
postal_df = pd.read_csv("http://cocl.us/Geospatial_data")
print(postal_df.shape)
postal_df.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


prepare for looking-up & setting lat,lng locations

In [114]:
df.reset_index(inplace=True)
postal_df.set_index("Postal Code", inplace=True)


lookup and set lat,lng

In [115]:

for i in range(0,df.shape[0]-1):
    postalCode = df.iloc[i].PostalCode
    try:
        [df.iloc[i,3], df.iloc[i,4]] = postal_df.loc[postalCode]
    except:
        print("{} lookup failed.", postalCode)


reverse what I did on index in preparation above

In [116]:
df.set_index('PostalCode', inplace=True)
postal_df.reset_index(inplace=True)

In [117]:
df.head()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Not assigned,43.662301,-79.389494


Assignment 3: cluster the neighborhoods in Toronto